# CSC311 Project's codebase

First, let's load the dataset

In [9]:
import pandas as pd
import numpy as np
import sklearn as sk
import plotly.express as px

In [10]:
import os
os.getcwd()
os.chdir('/Users/nguyenviet/Desktop/CSC311/group-project')  # Change to project root directory

In [11]:
df = pd.read_csv("data/training_data_clean.csv")
df.to_csv("./backup_training_data_clean.csv")
df.head(5)

,student_id,"In your own words, what kinds of tasks would you use this model for?",How likely are you to use this model for academic tasks?,Which types of tasks do you feel this model handles best? (Select all that apply.),"Based on your experience, how often has this model given you a response that felt suboptimal?",For which types of tasks do you feel this model tends to give suboptimal responses? (Select all that apply.),"Think of one task where this model gave you a suboptimal response. What did the response look like, and why did you find it suboptimal?",How often do you expect this model to provide responses with references or supporting evidence?,How often do you verify this model's responses?,"When you verify a response from this model, how do you usually go about it?",label
0,1,"General purpose tasks, like translating text, ...",3 — Neutral / Unsure,"Math computations,Data processing or analysis,...",3 — Sometimes,"Writing or debugging code,Explaining complex c...",I find that [THIS MODEL] has too much watering...,1 — Never,4 — Often,I double check with google searches or find fo...,ChatGPT
1,1,Coding,4 — Likely,Writing or debugging code,3 — Sometimes,"Math computations,Explaining complex concepts ...",Many times when i try to get [THIS MODEL] to d...,1 — Never,5 — Very often,I make sure that the code it writes is well wr...,Claude
2,1,"Math, coding sometimes",3 — Neutral / Unsure,"Math computations,Converting content between f...",4 — Often,"Data processing or analysis,Drafting professio...",[THIS MODEL] likes to put a lot of words and e...,2 — Rarely,3 — Sometimes,I make sure the facts its giving is correct. s...,Gemini
3,2,"General first case go-to, catch-all type of m...",5 — Very likely,"Explaining complex concepts simply,Converting ...",4 — Often,"Writing or debugging code,Data processing or a...","Gave it the main ideas I wanted included, but...",3 — Sometimes,2 — Rarely,Ask model to verify and doublecheck itself mu...,ChatGPT
4,2,Coding/debugging Problem-solving Coming up ...,4 — Likely,"Math computations,Writing or debugging code,Da...",4 — Often,"Data processing or analysis,Explaining complex...",Explain a complex concept Just restated the ...,2 — Rarely,1 — Never,#NAME?,Claude


In order to prevent the leakage of the data by the same user between the train, valid, and test subsets, we select the subsets of the student_ids to use for the splitting, not the records themselves

In [12]:
# This checks that all the users have all three records
assert all(df.groupby("student_id").size() == 3)

student_ids = df.student_id.unique()

In [13]:
SIZE_TOTAL = len(student_ids)
SIZE_TRAIN = int(SIZE_TOTAL * 0.6)
SIZE_VALID = int(SIZE_TOTAL * 0.2)
SIZE_TEST = SIZE_TOTAL - SIZE_TRAIN - SIZE_VALID

print(f'Total size: {SIZE_TOTAL}, training: {SIZE_TRAIN}, validation: {SIZE_VALID}, testing: {SIZE_TEST}')

Total size: 275, training: 165, validation: 55, testing: 55


In [14]:
rng = np.random.default_rng(42)
rng.shuffle(student_ids)
ids_train = student_ids[:SIZE_TRAIN]
ids_valid = student_ids[SIZE_TRAIN:SIZE_TRAIN + SIZE_VALID]
ids_test = student_ids[SIZE_TRAIN + SIZE_VALID:]
print(f"Student IDs used for training ({len(ids_train)} total): {ids_train}\n\n")
print(f"Student IDs used for validaiton ({len(ids_valid)} total): {ids_valid}\n\n")
print(f"Student IDs used for testing ({len(ids_test)} total): {ids_test}\n\n")

Student IDs used for training (165 total): [113  89  97 144 149  20  65  52  85 151 269 186  13 135 232  39 116 267
 257 101 273 134  51 237 216  27  69  25  63 215 187   4 139 160 175  47
 271 188 125  55 198 208 210  99  90 157 128 172 260  76 156 242 108 190
  71 236 256  81 133 211 150 166 229 147 254 159  83  48 255   8 127 207
 119 240 105  38  93 221 239 145 200  21 219 261  34   9 169 173 251  26
 263  30  78  80 275 102 163  67  10 146  60  96 182 250 131   5 180  49
 274  68 231  95 272 153  73 204 193 132  33   3 162  28 192 179  57   1
 122 189 268 109  82 136 170  56 154  53 199  87  29 225 107 217 121 141
 270 258 129 222 201  92 206  40 171 148  14 167 194   6  86 181 185 220
 214 197 174]


Student IDs used for validaiton (55 total): [ 79 247 120 249 115  84 124  46 104  91  19 205  88 226 112  74 243  22
 152  94 248  66 259 100  72 234 218 155 224 126 178 266 118  75 202  11
 117 227 158  64 184   2  43 123  18 168 264  54  77 233  41 209  70 244
 130]


Student IDs u

## Splitting the data

We used the following split of student IDs (should match the one in the previous cell's output):

```
Student IDs used for training (165 total): [113  89  97 144 149  20  65  52  85 151 269 186  13 135 232  39 116 267
 257 101 273 134  51 237 216  27  69  25  63 215 187   4 139 160 175  47
 271 188 125  55 198 208 210  99  90 157 128 172 260  76 156 242 108 190
  71 236 256  81 133 211 150 166 229 147 254 159  83  48 255   8 127 207
 119 240 105  38  93 221 239 145 200  21 219 261  34   9 169 173 251  26
 263  30  78  80 275 102 163  67  10 146  60  96 182 250 131   5 180  49
 274  68 231  95 272 153  73 204 193 132  33   3 162  28 192 179  57   1
 122 189 268 109  82 136 170  56 154  53 199  87  29 225 107 217 121 141
 270 258 129 222 201  92 206  40 171 148  14 167 194   6  86 181 185 220
 214 197 174]


Student IDs used for validaiton (55 total): [ 79 247 120 249 115  84 124  46 104  91  19 205  88 226 112  74 243  22
 152  94 248  66 259 100  72 234 218 155 224 126 178 266 118  75 202  11
 117 227 158  64 184   2  43 123  18 168 264  54  77 233  41 209  70 244
 130]


Student IDs used for testing (55 total): [223 106 230 245  12 143 213 212  44 253 161  62  58 235  24 165 246  16
 177  61 110 138 191  32 252  17  45 241 111 228 262 196  35  31  59 203
  36 183  50   7 103 176 140 265  42  23 164 114 238  15 142 195  37  98
 137]
```

In [15]:
train_data = df[df.student_id.isin(ids_train)]
valid_data = df[df.student_id.isin(ids_valid)]
test_data = df[df.student_id.isin(ids_test)]

Now we can save the split files:

In [ ]:
# train_data.to_csv("train_unprocessed.csv")
# valid_data.to_csv("valid_undrocessed.csv")
# test_data.to_csv("test_unprocessed.csv")

## Dataset overview

In [40]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 495 entries, 0 to 824
Data columns (total 11 columns):
 #   Column                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                   --------------  ----- 
 0   student_id                                                                                                                               495 non-null    int64 
 1   In your own words, what kinds of tasks would you use this model for?                                                                     493 non-null    object
 2   How likely are you to use this model for academic tasks?                                                                                 495 non-null    object
 3   Which types of tasks do you feel this model handles best? (Select all 

In [18]:
train_data.head(5)

,student_id,"In your own words, what kinds of tasks would you use this model for?",How likely are you to use this model for academic tasks?,Which types of tasks do you feel this model handles best? (Select all that apply.),"Based on your experience, how often has this model given you a response that felt suboptimal?",For which types of tasks do you feel this model tends to give suboptimal responses? (Select all that apply.),"Think of one task where this model gave you a suboptimal response. What did the response look like, and why did you find it suboptimal?",How often do you expect this model to provide responses with references or supporting evidence?,How often do you verify this model's responses?,"When you verify a response from this model, how do you usually go about it?",label
0,1,"General purpose tasks, like translating text, ...",3 — Neutral / Unsure,"Math computations,Data processing or analysis,...",3 — Sometimes,"Writing or debugging code,Explaining complex c...",I find that [THIS MODEL] has too much watering...,1 — Never,4 — Often,I double check with google searches or find fo...,ChatGPT
1,1,Coding,4 — Likely,Writing or debugging code,3 — Sometimes,"Math computations,Explaining complex concepts ...",Many times when i try to get [THIS MODEL] to d...,1 — Never,5 — Very often,I make sure that the code it writes is well wr...,Claude
2,1,"Math, coding sometimes",3 — Neutral / Unsure,"Math computations,Converting content between f...",4 — Often,"Data processing or analysis,Drafting professio...",[THIS MODEL] likes to put a lot of words and e...,2 — Rarely,3 — Sometimes,I make sure the facts its giving is correct. s...,Gemini
6,3,"Automation, Summarization, Idea Generation, Pr...",3 — Neutral / Unsure,"Writing or debugging code,Data processing or a...",4 — Often,"Math computations,Writing or debugging code,Co...","Having asked it to solve a numeric problem, it...",2 — Rarely,5 — Very often,"Trust nothing, check everything. Always see wh...",ChatGPT
7,3,I don't use it.,1 — Not at all likely,NaN,1 — Never,NaN,NaN,NaN,1 — Never,NaN,Claude


## Data preparation

Here we specify the pre-processing funciton that will be applied to all the splits of the data.

In [19]:
def map_colnames(df: pd.DataFrame) -> pd.DataFrame:
    colname_mapping = {
        'student_id': "student_id",
        'In your own words, what kinds of tasks would you use this model for?': "usages",
        'How likely are you to use this model for academic tasks?': "academic_use_frequency",
        'Which types of tasks do you feel this model handles best? (Select all that apply.)': "best_usages",
        'Based on your experience, how often has this model given you a response that felt suboptimal?': "bad_response_frequency",
        'For which types of tasks do you feel this model tends to give suboptimal responses? (Select all that apply.)': "worst_usages",
        'Think of one task where this model gave you a suboptimal response. What did the response look like, and why did you find it suboptimal?': "bad_response_example",
        'How often do you expect this model to provide responses with references or supporting evidence?': "references_frequency",
        "How often do you verify this model's responses?": "verification_frequency",
        'When you verify a response from this model, how do you usually go about it?': "verification_approach",
        'label': "label",
    }
    out_df = df.copy()
    out_df = df.rename(columns=colname_mapping)
    return out_df


In [20]:
def encode_ordinal_columns(df: pd.DataFrame) -> pd.DataFrame:
    def get_first_char(string):
        if not isinstance(string, str):
            return string
        return float(string[0])
    out_df = df.copy()
    out_df.academic_use_frequency = df.academic_use_frequency.apply(get_first_char).astype("Int16")
    out_df.bad_response_frequency = df.bad_response_frequency.apply(get_first_char).astype("Int16")
    out_df.references_frequency = df.references_frequency.apply(get_first_char).astype("Int16")
    out_df.verification_frequency = df.verification_frequency.apply(get_first_char).astype("Int16")
    return out_df

In [21]:
def normalize_ordinal_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    colnames_to_normalize = (
        "academic_use_frequency",
        "bad_response_frequency",
        "references_frequency",
        "verification_frequency",
    )
    for colname in colnames_to_normalize:
        mean = df[colname].mean()
        std = df[colname].std()
        df[colname] = (df[colname] - mean) / std
    return df

In [22]:
def decode_best_and_worst_usages(df: pd.DataFrame) -> pd.DataFrame:
    label_to_suffix = {
        "Math computations": "math",
        "Writing or debugging code": "code",
        "Data processing or analysis": "data",
        "Explaining complex concepts simply": "explanations",
        "Converting content between formats (e.g., LaTeX)": "conversion",
        "Writing or editing essays/reports": "essays",
        "Drafting professional text (e.g., emails, résumés)": "professional",
        "Brainstorming or generating creative ideas": "brainstorming",
    }
    label_to_best_colname = {k:f"best_usages_{v}" for k,v in label_to_suffix.items()}
    label_to_worst_colname = {k:f"worst_usages_{v}" for k,v in label_to_suffix.items()}

    out_df = df.copy()
    for label, colname in label_to_best_colname.items():
        out_df[colname] = df.best_usages.str.contains(label, regex=False).astype("Int16")
    for label, colname in label_to_worst_colname.items():
        out_df[colname] = df.worst_usages.str.contains(label, regex=False).astype("Int16")
    out_df = out_df.drop(columns=["best_usages", "worst_usages"])
    return out_df

In [23]:
def clean_dataset(df: pd.DataFrame, drop_na=False) -> pd.DataFrame:
    out_df = map_colnames(df)
    out_df = encode_ordinal_columns(out_df)
    out_df = decode_best_and_worst_usages(out_df)
    out_df = normalize_ordinal_columns(out_df)
    if drop_na:
        print(f'Warning: Dropping {out_df.isna().any(axis=1).sum()} ({out_df.isna().any(axis=1).sum() * 100 / out_df.shape[0]:.1f}%) records with missing values')
        out_df = out_df.dropna()
    return out_df

In [24]:
train_data_clean = clean_dataset(train_data, drop_na=True)

In [25]:
train_data_clean.describe()

,student_id,academic_use_frequency,bad_response_frequency,references_frequency,verification_frequency,best_usages_math,best_usages_code,best_usages_data,best_usages_explanations,best_usages_conversion,...,best_usages_professional,best_usages_brainstorming,worst_usages_math,worst_usages_code,worst_usages_data,worst_usages_explanations,worst_usages_conversion,worst_usages_essays,worst_usages_professional,worst_usages_brainstorming
count,435.000000,435.0,435.0,435.0,435.0,435.0,435.0,435.0,435.0,435.0,...,435.0,435.0,435.0,435.0,435.0,435.0,435.0,435.0,435.0,435.0
mean,135.234483,0.065292,0.052437,0.020775,0.047572,0.236782,0.494253,0.314943,0.586207,0.344828,...,0.501149,0.450575,0.604598,0.443678,0.381609,0.167816,0.222989,0.28046,0.2,0.268966
std,72.837425,0.980555,0.979392,0.98724,0.98335,0.425597,0.500543,0.465028,0.493079,0.475859,...,0.500574,0.498124,0.4895,0.49739,0.486341,0.374133,0.41673,0.449741,0.400461,0.443933
min,1.000000,-1.628035,-2.228557,-1.659109,-1.784779,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,77.000000,-0.831714,-1.048733,-0.753595,-0.964746,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,139.000000,-0.035392,0.131092,0.151918,-0.144712,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,193.000000,0.76093,0.131092,1.057432,0.675322,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
max,275.000000,1.557251,2.49074,1.962946,1.495356,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [26]:
train_data_clean

,student_id,usages,academic_use_frequency,bad_response_frequency,bad_response_example,references_frequency,verification_frequency,verification_approach,label,best_usages_math,...,best_usages_professional,best_usages_brainstorming,worst_usages_math,worst_usages_code,worst_usages_data,worst_usages_explanations,worst_usages_conversion,worst_usages_essays,worst_usages_professional,worst_usages_brainstorming
0,1,"General purpose tasks, like translating text, ...",-0.035392,0.131092,I find that [THIS MODEL] has too much watering...,-1.659109,0.675322,I double check with google searches or find fo...,ChatGPT,1,...,1,0,0,1,0,1,1,0,0,1
1,1,Coding,0.76093,0.131092,Many times when i try to get [THIS MODEL] to d...,-1.659109,1.495356,I make sure that the code it writes is well wr...,Claude,0,...,0,0,1,0,0,1,0,0,1,1
2,1,"Math, coding sometimes",-0.035392,1.310916,[THIS MODEL] likes to put a lot of words and e...,-0.753595,-0.144712,I make sure the facts its giving is correct. s...,Gemini,1,...,0,0,0,0,1,0,0,0,1,1
6,3,"Automation, Summarization, Idea Generation, Pr...",-0.035392,1.310916,"Having asked it to solve a numeric problem, it...",-0.753595,1.495356,"Trust nothing, check everything. Always see wh...",ChatGPT,0,...,1,1,1,1,0,0,1,0,0,1
9,4,"Summarization, inspiration.",-0.035392,1.310916,Debugging code. The response didn't cover much...,-0.753595,1.495356,I do some searching online to double check.,ChatGPT,0,...,0,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,271,Debugging: I’d use [THIS MODEL] explain trick...,0.76093,0.131092,"When working on larger coding projects, the mo...",1.057432,-0.964746,I usually verify a response by quickly Googlin...,ChatGPT,0,...,1,1,1,0,0,0,0,0,0,0
813,272,I used to use this for essay writing. I would ...,1.557251,0.131092,Fixing and debugging LaTeX code or converting ...,-0.753595,1.495356,I either ask it for sources where it got the r...,ChatGPT,1,...,0,0,0,1,0,0,1,1,1,0
816,273,"proofreading emails and essays, helping me und...",0.76093,1.310916,I have asked it to convert text to .md format....,1.057432,-0.144712,"If it is a computational question, I will try ...",ChatGPT,0,...,1,1,1,1,1,0,1,0,0,0
819,274,I would use this model for help with summarizi...,1.557251,-1.048733,I'm a math major and I often find that it can ...,0.151918,0.675322,It largely depends on the situation. But a com...,ChatGPT,0,...,1,0,1,0,1,0,0,0,0,0


In [27]:
# Stop words were modified from https://gist.github.com/sebleier/554280 (no, nor, don('t), and not, this were removed)
STOP_WORDS = (
    "i",
    "me",
    "my",
    "myself",
    "we",
    "our",
    "ours",
    "ourselves",
    "you",
    "your",
    "yours",
    "yourself",
    "yourselves",
    "he",
    "him",
    "his",
    "himself",
    "she",
    "her",
    "hers",
    "herself",
    "it",
    "its",
    "itself",
    "they",
    "them",
    "their",
    "theirs",
    "themselves",
    "what",
    "which",
    "who",
    "whom",
    "that",
    "these",
    "those",
    "am",
    "is",
    "are",
    "was",
    "were",
    "be",
    "been",
    "being",
    "have",
    "has",
    "had",
    "having",
    "do",
    "does",
    "did",
    "doing",
    "a",
    "an",
    "the",
    "and",
    "but",
    "if",
    "or",
    "because",
    "as",
    "until",
    "while",
    "of",
    "at",
    "by",
    "for",
    "with",
    "about",
    "against",
    "between",
    "into",
    "through",
    "during",
    "before",
    "after",
    "above",
    "below",
    "to",
    "from",
    "up",
    "down",
    "in",
    "out",
    "on",
    "off",
    "over",
    "under",
    "again",
    "further",
    "then",
    "once",
    "here",
    "there",
    "when",
    "where",
    "why",
    "how",
    "all",
    "any",
    "both",
    "each",
    "few",
    "more",
    "most",
    "other",
    "some",
    "such",
    "only",
    "own",
    "same",
    "so",
    "than",
    "too",
    "very",
    "s",
    "t",
    "can",
    "will",
    "just",
    "should",
    "now",
)

In [28]:
def alnum_lower_and_split(text: str) -> list[str]:
    if not isinstance(text, str):
        return []
    text = text.lower()
    text = "".join(c for c in text if c.isalnum() or c.isspace())
    text = text.replace("\n", " ").replace("\t", " ")
    return [word.rstrip('s') for word in text.split() if word not in STOP_WORDS]

In [29]:
from collections import Counter
from itertools import chain
text_corpus = "\n".join(chain(train_data_clean["usages"], train_data_clean["bad_response_example"], train_data_clean["verification_approach"]))
vocab_full = Counter(alnum_lower_and_split(text_corpus))
vocab = [word for word, count in vocab_full.most_common(500)]
# vocab

In [48]:
array = np.array([['A', 'B', 'C'],
                 ['D', 'E', 'F']])
array.shape

(2, 3)

In [52]:
# Manually coding this because we cannot use sklearn in the final code
def bag_of_words(series: pd.Series) -> np.typing.NDArray:
    cleaned_series = [alnum_lower_and_split(record) for record in series]
    out = np.zeros((series.shape[0], len(vocab)), dtype="int")
    for i, record in enumerate(cleaned_series):
        for j, word in enumerate(vocab):
            if word in record:
                out[i][j] = 1
    return out
for index, i in enumerate(bag_of_words(train_data_clean['usages'])[0]):
    if i:
        print(vocab[index])
print(train_data_clean['usages'][0])
bag_of_words(train_data_clean['usages'])

task
like
text
general
purpose
etc
General purpose tasks, like translating text, rewording text, etc


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], shape=(435, 500))

In [31]:
usages_bow = bag_of_words(train_data_clean['usages'])
usages_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], shape=(435, 500))

In [32]:
NON_BOW_COLUMNS = [
    "academic_use_frequency",
    "bad_response_frequency",
    "references_frequency",
    "verification_frequency",
    "best_usages_math",
    "best_usages_code",
    "best_usages_data",
    "best_usages_explanations",
    "best_usages_conversion",
    "best_usages_essays",
    "best_usages_professional",
    "best_usages_brainstorming",
    "worst_usages_math",
    "worst_usages_code",
    "worst_usages_data",
    "worst_usages_explanations",
    "worst_usages_conversion",
    "worst_usages_essays",
    "worst_usages_professional",
    "worst_usages_brainstorming",
]

In [33]:
BOW_COLUMNS = [
    "usages",
    "bad_response_example",
    "verification_approach",
]

In [34]:
TARGET_ENCODE = {
    "ChatGPT": 0,
    "Claude": 1,
    "Gemini": 2,
}
TARGET_DECODE = {v:k for k,v in TARGET_ENCODE.items()}

In [35]:
def prepare_features_and_targets(df: pd.DataFrame, drop_na=True, exclude_bow=False, encode_targets=False, one_hot_target_encoding=False) -> tuple[np.typing.NDArray, np.typing.NDArray]:
    clean_df = clean_dataset(df, drop_na)
    features = clean_df[NON_BOW_COLUMNS].to_numpy()
    if not exclude_bow:
        bow_features = []
        for col in BOW_COLUMNS:
            bow_features.append(bag_of_words(clean_df[col]))
        features = np.concat((features, *bow_features), axis=1)
    targets = clean_df["label"].to_numpy()
    if encode_targets:
        if one_hot_target_encoding:
            one_hot = np.zeros((targets.shape[0], 3))
            for i, t in enumerate(targets):
                one_hot[i][TARGET_ENCODE[t]] = 1
            targets = one_hot.astype(int)
        else:
            for i, t in enumerate(targets):
                targets[i] = TARGET_ENCODE[targets[i]]
            targets = targets.astype(int)
    return features.astype(float), targets

features, targets = prepare_features_and_targets(train_data, encode_targets=True, one_hot_target_encoding=True)
features, targets

(array([[-0.03539208,  0.13109157, -1.6591094 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.76092963,  0.13109157, -1.6591094 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.03539208,  1.3109157 , -0.75359547, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.76092963,  1.3109157 ,  1.05743238, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.55725133, -1.04873256,  0.15191845, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.55725133,  0.13109157, -0.75359547, ...,  0.        ,
          0.        ,  0.        ]], shape=(435, 1520)),
 array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        ...,
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]], shape=(435, 3)))

### Now let's save all the processed data

In [36]:
features_train_with_bow, targets_train_noenc = prepare_features_and_targets(train_data)
features_valid_with_bow, targets_valid_noenc = prepare_features_and_targets(valid_data)
features_test_with_bow, targets_test_noenc = prepare_features_and_targets(test_data)
features_train, targets_train_enc = prepare_features_and_targets(train_data, exclude_bow=True, encode_targets=True)
features_valid, targets_valid_enc = prepare_features_and_targets(valid_data, exclude_bow=True, encode_targets=True)
features_test, targets_test_enc = prepare_features_and_targets(test_data, exclude_bow=True, encode_targets=True)
_, targets_train_one_hot = prepare_features_and_targets(train_data, exclude_bow=True, encode_targets=True, one_hot_target_encoding=True)
_, targets_valid_one_hot = prepare_features_and_targets(valid_data, exclude_bow=True, encode_targets=True, one_hot_target_encoding=True)
_, targets_test_one_hot = prepare_features_and_targets(test_data, exclude_bow=True, encode_targets=True, one_hot_target_encoding=True)

In [37]:
import os
variables_to_filenames = {
    "data/train/features_train_with_bow.csv": features_train_with_bow,
    "data/train/targets_train_noenc.csv": targets_train_noenc,
    "data/valid/features_valid_with_bow.csv": features_valid_with_bow,
    "data/valid/targets_valid_noenc.csv": targets_valid_noenc,
    "data/test/features_test_with_bow.csv": features_test_with_bow,
    "data/test/targets_test_noenc.csv": targets_test_noenc,
    "data/train/features_train.csv": features_train,
    "data/train/targets_train_enc.csv": targets_train_enc,
    "data/valid/features_valid.csv": features_valid,
    "data/valid/targets_valid_enc.csv": targets_valid_enc,
    "data/test/features_test.csv": features_test,
    "data/test/targets_test_enc.csv": targets_test_enc,
    "data/train/targets_train_one_hot.csv": targets_train_one_hot,
    "data/valid/targets_valid_one_hot.csv": targets_valid_one_hot,
    "data/test/targets_test_one_hot.csv": targets_test_one_hot,
}
if not os.path.exists("data/"):
    os.mkdir("data/")
for filename, variable in variables_to_filenames.items():
    dirname = '/'.join(filename.split("/")[:-1])
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    pd.DataFrame(variable).to_csv(filename, index=False, header=False)

In [38]:
!tar -czvf datasets.tar.gz data/

a data
a data/training_data_clean.csv
a data/valid
a data/.DS_Store
a data/test
a data/train
a data/train/targets_train_enc.csv
a data/train/features_train.csv
a data/train/targets_train_noenc.csv
a data/train/features_train_with_bow.csv
a data/train/targets_train_one_hot.csv
a data/test/targets_test_one_hot.csv
a data/test/targets_test_enc.csv
a data/test/targets_test_noenc.csv
a data/test/features_test_with_bow.csv
a data/test/features_test.csv
a data/valid/features_valid_with_bow.csv
a data/valid/features_valid.csv
a data/valid/targets_valid_enc.csv
a data/valid/targets_valid_one_hot.csv
a data/valid/targets_valid_noenc.csv


In [39]:
!cat data/train/features_train.csv

-0.035392075612731194,0.13109156972179992,-1.6591094000631967,0.6753219255929941,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
0.7609296256737236,0.13109156972179992,-1.6591094000631967,1.4953556923844868,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
-0.035392075612731194,1.3109156972179987,-0.7535954744865363,-0.14471184119849856,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
-0.035392075612731194,1.3109156972179987,-0.7535954744865363,1.4953556923844868,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
-0.035392075612731194,1.3109156972179987,-0.7535954744865363,1.4953556923844868,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
0.7609296256737236,0.13109156972179992,0.151918451090124,0.6753219255929941,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
-0.8317137768991859,-1.0487325577743989,0.151918451090124,0.6753219255929941,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.